# GraphRAG Patterns with Neo4j

This notebook demonstrates how to use Neo4j's GraphRAG library to implement retrieval-augmented generation with graph context, starting with simple patterns and building up to more complex ones.

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.indexes import create_vector_index
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever
from neo4j_graphrag import GraphRAG, RagTemplate

# Load environment variables
load_dotenv()

# Initialize Neo4j connection
URI = os.getenv('NEO4J_URI')
AUTH = (os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Create Neo4j driver
driver = GraphDatabase.driver(URI, auth=AUTH)

# Initialize LLM and embeddings
llm = OpenAILLM(
    model_name="gpt-4",
    model_params={
        "temperature": 0  # Lower temperature for more deterministic results
    }
)

embedder = OpenAIEmbeddings()

## Create Vector Index

First, let's create a vector index for our documents:

In [ ]:
# Create vector index for document embeddings
create_vector_index(
    driver,
    name="document_embeddings",
    label="Document",
    embedding_property="embedding",
    dimensions=1536,
    similarity_fn="cosine"
)

## Pattern 1: Simple Vector Search

Let's start with basic vector similarity search:

In [ ]:
# Initialize basic vector retriever
vector_retriever = VectorRetriever(
    driver,
    index_name="document_embeddings",
    embedder=embedder,
    return_properties=["text"]
)

# Create RAG template
rag_template = RagTemplate(
    template="""Answer the Question using the following Context. Only respond with information mentioned in the Context.

# Question:
{query_text}

# Context:
{context}

# Answer:
""",
    expected_inputs=['query_text', 'context']
)

# Initialize GraphRAG with vector retriever
vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)

# Example query
question = "What are the key features of our laptop products?"
result = vector_rag.search(question, retriever_config={'top_k': 3})
print(f"Answer: {result.answer}")

## Pattern 2: Simple Graph Context

Now let's add basic graph context - just looking at directly connected nodes:

In [ ]:
# Initialize simple graph retriever
simple_graph_retriever = VectorCypherRetriever(
    driver,
    index_name="document_embeddings",
    embedder=embedder,
    retrieval_query="""
    // Start with similar documents
    WITH node AS doc
    
    // Get directly connected documents
    OPTIONAL MATCH (doc)-[:ABOUT]->(p:Product)
    
    // Return document text
    RETURN doc.text + '\nProduct: ' + p.name as info
    """
)

# Initialize GraphRAG with simple graph retriever
simple_graph_rag = GraphRAG(llm=llm, retriever=simple_graph_retriever, prompt_template=rag_template)

# Example query
question = "Tell me about the Laptop Pro model."
result = simple_graph_rag.search(question, retriever_config={'top_k': 3})
print(f"Answer: {result.answer}")

## Pattern 3: Adding Support Cases

Let's extend our graph context to include support cases:

In [ ]:
# Initialize support case retriever
support_retriever = VectorCypherRetriever(
    driver,
    index_name="document_embeddings",
    embedder=embedder,
    retrieval_query="""
    // Start with similar documents
    WITH node AS doc
    
    // Get product and its support cases
    MATCH (doc)-[:ABOUT]->(p:Product)
    OPTIONAL MATCH (p)<-[:ABOUT]-(support:Document)
    WHERE support.type = 'support_case'
    
    // Return combined information
    RETURN 
        doc.text + 
        '\nProduct: ' + p.name + 
        CASE WHEN support IS NOT NULL 
            THEN '\nSupport Case: ' + support.text
            ELSE ''
        END as info
    """
)

# Initialize GraphRAG with support retriever
support_rag = GraphRAG(llm=llm, retriever=support_retriever, prompt_template=rag_template)

# Example query
question = "What issues have users reported with the Laptop Pro?"
result = support_rag.search(question, retriever_config={'top_k': 3})
print(f"Answer: {result.answer}")

## Pattern 4: Category-Based Context

Finally, let's look at products in the same category:

In [ ]:
# Initialize category-based retriever
category_retriever = VectorCypherRetriever(
    driver,
    index_name="document_embeddings",
    embedder=embedder,
    retrieval_query="""
    // Start with similar documents
    WITH node AS doc
    
    // Get product and category
    MATCH (doc)-[:ABOUT]->(p:Product)-[:IN_CATEGORY]->(cat:Category)
    
    // Get other products in same category
    WITH doc, p, cat
    MATCH (cat)<-[:IN_CATEGORY]-(related:Product)
    WHERE related <> p
    
    // Return information
    RETURN 
        doc.text + 
        '\nProduct: ' + p.name +
        '\nCategory: ' + cat.name +
        '\nRelated Products: ' + collect(related.name)[0..3] as info
    """
)

# Initialize GraphRAG with category retriever
category_rag = GraphRAG(llm=llm, retriever=category_retriever, prompt_template=rag_template)

# Example query
question = "What other laptops are similar to the Laptop Pro?"
result = category_rag.search(question, retriever_config={'top_k': 3})
print(f"Answer: {result.answer}")

## Key GraphRAG Features Used

1. **Vector Search**: Basic similarity search using embeddings
2. **Simple Graph Context**: Direct relationships to products
3. **Support Cases**: Including user-reported issues
4. **Category Context**: Finding related products

Each pattern builds on the previous one, gradually adding more graph context while keeping the queries focused and readable.